In [ ]:
import csv          # Python package for reading and writing CSV files.
import copy         # Copy data structures.
import json

import sys,os

# You can change to wherever you want to place your CSV files.
rel_path = os.path.realpath('./data')
rel_path += '/'

In [ ]:
class CSVTable():

    # Change to wherever you want to save the CSV files.

    def __init__(self, table_name, table_file, key_columns):
        '''
        Constructor
        :param table_name: Logical names for the data table.
        :param table_file: File name of CSV file to read/write.
        :param key_columns: List of column names the form the primary key.
        '''
        self.table_name = table_name
        self.table_file = rel_path + table_file
        self.key_columns = key_columns
        self.data = []
    
    def __str__(self):
        '''
        Pretty print the table and state.
        :return: String
        '''
        return json.dumps(self.data, indent=2)
        
        
        
    def load(self):
        '''
        Load information from CSV file.
        :return: None
        '''
        try:
            with open(self.table_file, mode = 'r') as csv_file:
                csv_reader = csv.DictReader(csv_file)
                for i in csv_reader:
                    self.data.append(i)
        except IOError:
            print ("The datatable file doens't exit")
        
        for i in self.key_columns:
            if i not in self.data[0]:
                raise ValueError("Keys in primary key_columns doesn't match the database")

    def find_by_primary_key(self, string_set, fields=None):
        
        for i in string_set:
            if i not in self.data[0]:
                raise ValueError("primary_keys in string_set doesn't match the database")

        if fields == None:
            for i in self.data:
                count_match = 0
                for j in range(string_set):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    return i
        else:
            unwanted = self.data[0].keys() - fields
            for i in self.data:
                count_match = 0
                for j in range(string_set):
                    if i[self.key_columns[j]] == string_set[j]:
                        count_match += 1
                if count_match == len(self.key_columns):
                    for unwanted_key in unwanted:
                        del i[unwanted_key]
                    return i

        return None        

    def find_by_template(self, t, fields=None):
        '''
        Return a table containing the rows matching the template and field selector.
        :param t: Template that the rows much match.
        :param fields: A list of columns to include in responses.
        :return: CSVTable containing the answer.
        '''
        t_keys = t.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")


        result = []
        if fields == None:
            keys = t.keys()
            for i in self.data:
                count_match = 0
                for key in keys:
                    if i[key] == t[key]:
                        count_match += 1
                
                if count_match == len(keys):
                    result.append(i)
        else:
            keys = t.keys()
            unwanted = self.data[0].keys() - fields
            for i in self.data:
                count_match = 0
                for key in keys:
                    if i[key] == t[key]:
                        count_match += 1
                
                if count_match == len(keys):
                    for unwanted_key in unwanted:
                        del i[unwanted_key]
                    result.append(i)
        
        return result

    def save(self):
        '''
        Write updated CSV back to the original file location.
        :return: None
        '''
        try:
            with open(self.table_file, mode='w', newline = '') as csv_file:
                fieldnames = self.data[0].keys()
                writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
                writer.writeheader()
                for i in self.data:
                    writer.writerow(i)
        except IOError:
            print( "Can't open the file to write" )

    def insert(self, r):
        '''
        Insert a new row into the table.
        :param r: New row.
        :return: None. Table state is updated.
        '''
        t_keys = r.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")

        # if (  ) insert duplicated primary keys

        self.data.append(r)


    def delete(self, t):
        '''
        Delete all tuples matching the template.
        :param t: Template
        :return: None. Table is updated.
        '''
        t_keys = t.keys()
        for i in t_keys:
            if i not in self.data[0]:
                raise ValueError("Keys in templated doesn't match the database format")
        
        keys = t.keys()
        for i in self.data:
            count_match = 0
            for key in keys:
                if i[key] == t[key]:
                    count_match += 1
            
            if count_match == len(keys):
                self.data.remove(i)

In [ ]:
def test1():

    csvt = CSVTable("People", "PeopleSmall.csv", ["playerID"])
    csvt.load()
    print("Table = ", csvt)


def test_template(test_name, table_name, table_file, key_columns, template, fields=None, show_rows=False):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Template = ", template)
    print("Fields = ", fields)

    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()

        if not show_rows:
            print("Table name = ", csvt.table_name)
            print("Table file = ", csvt.table_file)
            print("Table keys = ", csvt.key_columns)
        else:
            print(csvt)

        r = csvt.find_by_template(template, fields)
        print("Result table:")
        print(json.dumps(r, indent=2))
        
    except ValueError as ve:
        print("Exception = ", ve)


def test_insert(test_name, table_name, table_file, key_columns, row, show_rows=False):
    print("\n\n*******************************")
    print("Test name = ", test_name)
    print("Row to insert = ", row)

    try:
        csvt = CSVTable(table_name, table_file, key_columns)
        csvt.load()

        if not show_rows:
            print("Table name = ", csvt.table_name)
            print("Table file = ", csvt.table_file)
            print("Table keys = ", csvt.key_columns)
        else:
            print(csvt)

        r = csvt.insert(row)
        print("Result table:")

        csvt.save()

        if show_rows:
            print(json.dumps(r, indent=2))

    except ValueError as ve:
        print("Exception = ", ve)


def test_templates():
    test_template("Test2", "People", "People.csv", ["playerID"],
                  {"birthMonth": "9", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test3", "People", "People.csv", ["playerID"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test4", "People", "People.csv", ["canary"],
                  {"nameFirst": "Ted", "nameLast": "Williams"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    test_template("Test5", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test6", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "iq": 100}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test7", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1961"}, ["playerID", "yearID", "teamID", "AB", "H", "HR"],
                  False)

    test_template("Test7", "Batting", "Batting.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "willite01", "yearID": "1960"}, ["playerID", "yearID", "teamID", "AB", "H", "HR", "Age"],
                  False)


def test_inserts():

    test_insert("Insert Test 1", "People", "PeopleSmall.csv", ["playerID"],
                {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
                False)

    test_template("Find after insert 1", "People", "PeopleSmall.csv", ["playerID"],
                  {"nameLast": "Ferguson"}, ["nameLast", "nameFirst", "birthMonth", "birthYear"],
                  False)

    try:
        test_insert("Insert Test 2", "People", "PeopleSmall.csv", ["playerID"],
                    {"playerID": "dff1", "nameLast": "Ferguson", "nameFirst": "Donald"},
                    False)

        raise ValueError("That insert should not have worked!")

    except ValueError as ve:
        print("OK. Did not insert duplicate key.")


    test_insert("Insert Test 3", "Batting", "BattingSmall.csv", ["playerID", "yearID", "teamID", "stint"],
                {"playerID": "dff1", "teamID": "BOS", "yearID": "2018", "stint": "1",
                    "AB": "100", "H": "100"},
                False)

    test_template("Find after insert 3", "Batting", "BattingSmall.csv", ["playerID", "yearID", "teamID", "stint"],
                  {"playerID": "dff1"}, None,
                  False)


In [52]:
import pymysql
import json

In [60]:
def templateToWhereClause(t):
    s = ""
    for k,v in t.items():
        if s != "":
            s += " AND "
        s += k + "='" + v + "'"

    if s != "":
        s = "WHERE " + s;

    return s
def templateToInsertClause(t):
    s = "("
    keys = t.keys()
    count = 0
    for i in keys:
        s += i
        count += 1
        if count != len(keys):
            s += ","

    s += ") VALUES ("
    count = 0
    for i in keys:
        s += "'" + t[i] + "'"
        count += 1
        if count != len(keys): 
            s += ","
    s+= ")"

    return s

In [65]:
class RDBDataTable():
    def __init__(self, host, user, password, db_name, table_name ):
        # Your code goes here
        cnx = pymysql.connect(host= host,
                              user= user,
                              password= password,
                              db=db_name,
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)
        self.cursor = cnx.cursor()
        self.table_name = table_name

    def find_by_primary_key(self, string_set, fields=None):
        q = ""
        self.cursor.execute(q)
        r = self.cursor.fetchall()
        return r

    # The input is:
    # t: The template to match. The result is a list of rows
    # whose attribute/value pairs exactly match the template.
    # fields: A subset of the fields to include for each result.
    # Raises an exception if the template or list of fields contains
    # a column/attribute name not in the file.
    def find_by_template(self, t, fields=None):
        w = templateToWhereClause(t)

        if fields == None:
            q = "SELECT * FROM " + self.table_name + " " + w + ";"
        else:
            fields_string = ""
            for i in fields:
                fields_string += i + ","
            fields_string = fields_string[:-1]

            q = "SELECT " + fields_string + " FROM " + self.table_name + " " + w + ";"

        print ("Query = ", q)
        self.cursor.execute(q)
        r = self.cursor.fetchall()
        return r


    # Inserts the row into the table. 
    # Raises on duplicate key or invalid columns.
    def insert(self, t):
        w = templateToInsertClause(t)
        q = "INSERT INTO " + self.table_name + " " + w + ";"
        print("Query =", q)
        self.cursor.execute(q)


       
    # t: A template.
    # Deletes all rows matching the template.
    def delete(self, t):
        w = templateToWhereClause(t)
        q = "DELETE FROM " + self.table_name + " " + w + ";"
        print("Query =", q)
        self.cursor.execute(q)

        

In [66]:
input = { "birthMonth": "9", "nameLast": "Williams"}
rdbt = RDBDataTable('localhost','root', 'zhangchi25','hw1','People')
r = rdbt.find_by_template(input, ["nameLast", "nameFirst", "birthMonth", "birthYear"])
r

Query =  SELECT nameLast,nameFirst,birthMonth,birthYear FROM People WHERE birthMonth='9' AND nameLast='Williams';


[{'nameLast': 'Williams',
  'nameFirst': 'Bernie',
  'birthMonth': '9',
  'birthYear': '1968'},
 {'nameLast': 'Williams',
  'nameFirst': 'Don',
  'birthMonth': '9',
  'birthYear': '1931'},
 {'nameLast': 'Williams',
  'nameFirst': 'Don',
  'birthMonth': '9',
  'birthYear': '1935'},
 {'nameLast': 'Williams',
  'nameFirst': 'Nick',
  'birthMonth': '9',
  'birthYear': '1993'},
 {'nameLast': 'Williams',
  'nameFirst': 'Randy',
  'birthMonth': '9',
  'birthYear': '1975'},
 {'nameLast': 'Williams',
  'nameFirst': 'Stan',
  'birthMonth': '9',
  'birthYear': '1936'}]

In [61]:
templateToWhereClause(input)

"WHERE birthMonth='9' AND nameLast='Williams'"